In [1]:
# Install required logging tools
%pip install -q dagshub 
%pip install -U mlflow

# Initialize DagsHub MLflow integration
import dagshub
dagshub.init(repo_owner='AleksandreBakhtadze', repo_name='ML-FinalProject-Walmart-Recruiting---Store-Sales-Forecasting', mlflow=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires dacite>=1.8, but you have dacite 1.6.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=1b45fe80-3535-40e4-a101-c9126b3514bd&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=91161416661c9929f1624edbd0399d8c9a4b5e411703f3c635b84122995b32c8




Output()

Accessing as AleksandreBakhtadze

Initialized MLflow to track repo "AleksandreBakhtadze/ML-FinalProject-Walmart-Recruiting---Store-Sales-Forecasting"

Repository AleksandreBakhtadze/ML-FinalProject-Walmart-Recruiting---Store-Sales-Forecasting initialized!

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv
/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip


In [3]:
import pandas as pd
import numpy as np
import zipfile
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import mlflow
import mlflow.sklearn
import joblib
import os

In [4]:
# Helper function to read zipped CSVs
def read_zipped_csv(path):
    with zipfile.ZipFile(path) as z:
        file_name = z.namelist()[0]
        return pd.read_csv(z.open(file_name))

# Load data
train_df = read_zipped_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
features_df = read_zipped_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip')
stores_df = pd.read_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv')
test_df = read_zipped_csv('/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip')

In [5]:
def merge_datasets(X, features_df, stores_df):
    df = X.copy()
    df_full = df.merge(features_df, on=["Store", "Date"], how="left")
    df_full = df_full.merge(stores_df, on="Store", how="left")
    df_full = df_full.drop(columns=['IsHoliday_y'])
    df_full = df_full.rename(columns={'IsHoliday_x': 'IsHoliday'})
    return df_full

def add_markdown_indicators(df):
    df = df.copy()
    for i in range(1, 6):
        df[f'MarkDown{i}_Missing'] = df[f'MarkDown{i}'].isnull().astype(int)
        print(f"Created column: MarkDown{i}_Missing")
    return df

def feature_engineering(df):
    df = df.copy()
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Week'] = df['Date'].dt.isocalendar().week.astype(int)
    df['Day'] = df['Date'].dt.day
    df['IsHoliday'] = df['IsHoliday'].astype(int)
    df['Type'] = df['Type'].map({'A': 0, 'B': 1, 'C': 2})
    df = add_markdown_indicators(df)
    return df

def convert_date(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    return df

class ForwardFillImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        return X.ffill()

# Custom transformer to handle merging with external data
class DatasetMerger(BaseEstimator, TransformerMixin):
    def __init__(self, features_df, stores_df):
        self.features_df = features_df
        self.stores_df = stores_df
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return merge_datasets(X, self.features_df, self.stores_df)


In [6]:
# Convert features_df['Date'] to datetime (auxiliary data, not test set)
try:
    features_df['Date'] = pd.to_datetime(features_df['Date'], format='%Y-%m-%d')
except Exception as e:
    print(f"features_df['Date'] conversion failed: {str(e)}")
    print("features_df['Date'].head():", features_df['Date'].head().tolist())
    raise

# Verify dtypes
print("train_df['Date'].dtype:", train_df['Date'].dtype)
print("features_df['Date'].dtype:", features_df['Date'].dtype)
print("test_df['Date'].dtype:", test_df['Date'].dtype)

train_df['Date'].dtype: object
features_df['Date'].dtype: datetime64[ns]
test_df['Date'].dtype: object


In [7]:
# Feature columns
feature_cols = [
    'Store', 'Dept', 'Type', 'Size',
    'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
    'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5',
    'MarkDown1_Missing', 'MarkDown2_Missing', 'MarkDown3_Missing', 'MarkDown4_Missing', 'MarkDown5_Missing',
    'IsHoliday', 'Year', 'Month', 'Week'
]

In [8]:
# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num_ffill', ForwardFillImputer(), ['CPI', 'Unemployment']),
        ('markdown_fill', SimpleImputer(strategy='constant', fill_value=0), 
         ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']),
        ('scale', StandardScaler(), ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Size']),
        ('passthrough', 'passthrough', 
         ['Store', 'Dept', 'Type', 'IsHoliday', 'Year', 'Month', 'Week',
          'MarkDown1_Missing', 'MarkDown2_Missing', 'MarkDown3_Missing', 'MarkDown4_Missing', 'MarkDown5_Missing'])
    ]
)


In [9]:
# Pipeline with all preprocessing steps
pipeline = Pipeline([
    ('convert_date', FunctionTransformer(convert_date, validate=False)),
    ('merge', DatasetMerger(features_df=features_df, stores_df=stores_df)),
    ('feature_engineering', FunctionTransformer(feature_engineering, validate=False)),
    ('preprocess', preprocessor),
    ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, n_jobs=-1))
])

In [10]:
# Prepare training data
train_full = train_df.copy()
y = train_full['Weekly_Sales']
train_full = train_full.drop(columns=['Weekly_Sales'])

# Convert Date for splitting only (minimal preprocessing for training)
try:
    train_full_for_split = train_full.copy()
    train_full_for_split['Date'] = pd.to_datetime(train_full_for_split['Date'], format='%Y-%m-%d')
except Exception as e:
    print(f"train_full['Date'] conversion failed: {str(e)}")
    print("train_full['Date'].head():", train_full['Date'].head().tolist())
    raise

print("Columns in train_full before split:", train_full.columns.tolist())
print("train_full['Date'].dtype:", train_full['Date'].dtype)
print("train_full_for_split['Date'].dtype:", train_full_for_split['Date'].dtype)

# Split data
train_full_for_split = train_full_for_split.sort_values('Date')
split_date = train_full_for_split['Date'].quantile(0.9)
train_idx = train_full_for_split['Date'] < split_date
X_train = train_full[train_idx]
y_train = np.clip(y[train_idx], 0, y.quantile(0.99))
X_val = train_full[~train_idx]
y_val = y[~train_idx]
is_holiday_val = X_val['IsHoliday']
print("Columns in X_train:", X_train.columns.tolist())

# Verify columns after pipeline transformation
X_train_transformed = pipeline.named_steps['convert_date'].transform(X_train)
X_train_transformed = pipeline.named_steps['merge'].transform(X_train_transformed)
X_train_transformed = pipeline.named_steps['feature_engineering'].transform(X_train_transformed)
print("Columns in X_train after pipeline transformations:", X_train_transformed.columns.tolist())


Columns in train_full before split: ['Store', 'Dept', 'Date', 'IsHoliday']
train_full['Date'].dtype: object
train_full_for_split['Date'].dtype: datetime64[ns]


/tmp/ipykernel_13/1116745263.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_train = train_full[train_idx]
/tmp/ipykernel_13/1116745263.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_val = train_full[~train_idx]


Columns in X_train: ['Store', 'Dept', 'Date', 'IsHoliday']
Created column: MarkDown1_Missing
Created column: MarkDown2_Missing
Created column: MarkDown3_Missing
Created column: MarkDown4_Missing
Created column: MarkDown5_Missing
Columns in X_train after pipeline transformations: ['Store', 'Dept', 'Date', 'IsHoliday', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'Year', 'Month', 'Week', 'Day', 'MarkDown1_Missing', 'MarkDown2_Missing', 'MarkDown3_Missing', 'MarkDown4_Missing', 'MarkDown5_Missing']


In [11]:
# WMAE metric
def calculate_wmae(y_true, y_pred, is_holiday):
    weights = np.where(is_holiday, 5, 1)
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

In [12]:
# MLflow setup
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/AleksandreBakhtadze/ML-FinalProject-Walmart-Recruiting---Store-Sales-Forecasting.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'AleksandreBakhtadze'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '034b77b38fbceb0a45865e04299f524469d930d4'

In [13]:
mlflow.set_experiment("XGBoost_Improved_Pipeline")
# End any active MLflow run
mlflow.end_run()

with mlflow.start_run(run_name="XGBoost_Improved_Training"):
    # Fit pipeline
    pipeline.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)
    wmae = calculate_wmae(y_val, y_pred, is_holiday_val)
    print(f"Validation MAE: {mae:.2f}")
    print(f"Validation WMAE: {wmae:.2f}")
    
    # Log parameters and metrics
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("random_state", 42)
    mlflow.log_metric("val_mae", mae)
    mlflow.log_metric("val_wmae", wmae)
    
    # Attempt to save model to MLflow model registry
    model_name = "XGBoost_Pipeline_Model"
    try:
        mlflow.sklearn.log_model(pipeline, "model", registered_model_name=model_name)
        print(f"Model successfully registered in MLflow Model Registry as {model_name}")
    except Exception as e:
        print(f"Failed to register model in MLflow Model Registry: {str(e)}")
        # Fallback to saving as joblib artifact
        os.makedirs("/kaggle/working/models", exist_ok=True)
        model_path = "/kaggle/working/models/xgboost_pipeline_improved.joblib"
        joblib.dump(pipeline, model_path)
        mlflow.log_artifact(model_path)
        print(f"Model saved as artifact: {model_path}")


Created column: MarkDown1_Missing
Created column: MarkDown2_Missing
Created column: MarkDown3_Missing
Created column: MarkDown4_Missing
Created column: MarkDown5_Missing
Created column: MarkDown1_Missing
Created column: MarkDown2_Missing
Created column: MarkDown3_Missing
Created column: MarkDown4_Missing
Created column: MarkDown5_Missing
Validation MAE: 3784.87
Validation WMAE: 3870.47


2025/07/06 17:35:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Failed to register model in MLflow Model Registry: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
Model saved as artifact: /kaggle/working/models/xgboost_pipeline_improved.joblib
🏃 View run XGBoost_Improved_Training at: https://dagshub.com/AleksandreBakhtadze/ML-FinalProject-Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/4/runs/0011b756a4674eaba6d476b9b767123a
🧪 View experiment at: https://dagshub.com/AleksandreBakhtadze/ML-FinalProject-Walmart-Recruiting---Store-Sales-Forecasting.mlflow/#/experiments/4


In [14]:
# Test predictions
test_full = test_df.copy()
print("Columns in test_full before pipeline:", test_full.columns.tolist())
print("test_full['Date'].dtype:", test_full['Date'].dtype)
test_preds = pipeline.predict(test_full)

Columns in test_full before pipeline: ['Store', 'Dept', 'Date', 'IsHoliday']
test_full['Date'].dtype: object
Created column: MarkDown1_Missing
Created column: MarkDown2_Missing
Created column: MarkDown3_Missing
Created column: MarkDown4_Missing
Created column: MarkDown5_Missing


In [15]:
# Submission
test_full_transformed = pipeline.named_steps['convert_date'].transform(test_full)
test_full_transformed = pipeline.named_steps['merge'].transform(test_full_transformed)
test_full_transformed = pipeline.named_steps['feature_engineering'].transform(test_full_transformed)
test_full_transformed['Id'] = test_full_transformed['Store'].astype(str) + '_' + test_full_transformed['Dept'].astype(str) + '_' + test_full_transformed['Date'].dt.strftime('%Y-%m-%d')
submission_df = pd.DataFrame({'Id': test_full_transformed['Id'], 'Weekly_Sales': test_preds})
submission_df.to_csv("submission.csv", index=False)
mlflow.log_artifact("submission.csv")

Created column: MarkDown1_Missing
Created column: MarkDown2_Missing
Created column: MarkDown3_Missing
Created column: MarkDown4_Missing
Created column: MarkDown5_Missing
